In [57]:
using JuMP, CPLEX, LinearAlgebra, DelimitedFiles, Plots, LinA
plotly()
mc = readdlm("mcSmashWiki2.csv", ';', Float64)
mc= round.(mc, digits=2)
usage = [readdlm("communityDistribution.csv", ',', Float64)...];


In [58]:
#Basic Model

model = Model(CPLEX.Optimizer)
MOI.set(model, MOI.Silent(), true)
@variable model x[1:26] >= 0

@objective model Max x'*mc*usage
@constraint model sum(x) == 1
optimize!(model)
println(value.(x)')

objective_value(model)

[0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]


0.533512629577656

In [68]:
model = Model(CPLEX.Optimizer)
MOI.set(model, MOI.Silent(), true)
@variable model x[1:26] >= 0

σ(x) = 1 /(1 +exp(-5x))
learningRate(x) = 2σ(x)-1
lrLin = Linearize(learningRate,0,1,Relative(1))
nbpiece=length(lrLin)

@variable model active[1:26,1:nbpiece] binary = true
@variable model f[1:26,1:nbpiece] >= 0

#only one piece has weight
for i in 1:26
    @constraint model sum(active[i,:]) <= 1
end

#binding piece with activation
for i=1:26,j=1:nbpiece
    @constraint model f[i,j] - lrLin[j].xMin*active[i,j] >= 0
    @constraint model f[i,j] - lrLin[j].xMax*active[i,j] <= 0
end

#constraint probability distribution

@constraint model sum(f) == 1

#function to optimize
α = 0
for i=1:26,j=1:nbpiece    
    α += (mc*usage)[i]*(lrLin[j].a*f[i,j] + lrLin[j].b*active[i,j])
end

@objective model Max α

optimize!(model)

playrates = sum(value.(f),dims=2)

26×1 Array{Float64,2}:
 0.06967701380941513
 0.0
 0.2227834684839847
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.10107707395808575
 0.0
 0.0
 0.10107707395808575
 0.0
 0.10107707395808575
 0.10107707395808575
 0.10107707395808575
 0.0
 0.0
 0.10107707395808575
 0.10107707395808575
 0.0
 0.0
 0.0
 0.0
 0.0

In [66]:
objective_value(model)

0.5388530909997286

In [67]:
bar(playrates,legend=false,ylabel="Utilisation %")

<!DOCTYPE html>
 
 
 Plots.jl

In [61]:
plot([learningRate x->x],0,1,label=["Lr" "Linear"],legend=:bottomright,lw = 3)

<!DOCTYPE html>
 
 
 Plots.jl

In [62]:
plot([x->lrLin(x) x->x],0,1,label=["LrLin" "Linear"],lw = 3,legend=:bottomright)

<!DOCTYPE html>
 
 
 Plots.jl